<a href="https://colab.research.google.com/github/antsfly626/AI-Publications-Summary/blob/main/chatbot_time_let's_go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("API key loaded successfully!")


API key loaded successfully!


In [3]:
"The study found that strawberry yields were highest when planted in late March and harvested in early June. Temperatures between 18°C and 25°C yielded optimal sugar content."

from openai import OpenAI
from google.colab import userdata

# Access the API key from Colab secrets
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

text = """
The study found that strawberry yields were highest when planted in late March and harvested in early June.
Temperatures between 18°C and 25°C yielded optimal sugar content.
"""

question = "What months are the best to grow strawberries in?"

prompt = f"""
You are a scientific assistant.
Here is the research text:

{text}

Answer the question using only the information above:
{question}
"""

response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Late March through early June — i.e., planting in late March and harvesting in early June.


In [4]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from google.colab import userdata

template = """
You are a helpful research assistant.
Use the context below to answer the question accurately.

Context:
{context}

Question:
{question}
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=userdata.get('OPENAI_API_KEY'))

chain = LLMChain(llm=llm, prompt=prompt)

result = chain.run(context=text, question="What is the optimal temperature?")
print(result)

/tmp/ipython-input-421665100.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0, openai_api_key=userdata.get('OPENAI_API_KEY'))
/tmp/ipython-input-421665100.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-421665100.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(context=text, question="What is the optimal temperature?"

The optimal temperature for achieving the highest sugar content in strawberries, according to the study, is between 18°C and 25°C.


In [1]:
!pip install langchain faiss-cpu openai tiktoken langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from google.colab import userdata

In [7]:
# split into chonks

text = """
The study found that strawberry yields were highest when planted in late March and harvested in early June.
Temperatures between 18°C and 25°C yielded optimal sugar content.
In Minnesota, harvesting every other day is typical unless hot weather or spotted wing drosophila (SWD) infestations necessitate daily harvest.
Research has found that harvesting berries every 1 to 2 days rather than every 3 days reduces SWD infection and increases marketable yield.
More frequent harvest also decreases the likelihood of fruit rots like botrytis gray mold.
The duration of the harvest, and how often berries are picked, also depend on weather conditions, variety, soil factors, irrigation,
and the health of the plants.
Plants that are irrigated and fertilized properly through fertigation or granular fertilizers will produce higher daily yields as fruit
ripens more quickly and berries are larger in size. This calls for more frequent harvests.
"""

# split long text into smaller overlapping chonks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([text])
print(f"Split into {len(docs)} chunks")

Split into 2 chunks


In [8]:
# embed for chonks
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

db = FAISS.from_documents(docs, embeddings)
# we want top 3 most relevant chunks
retriever = db.as_retriever(search_kwargs={"k": 3})

/tmp/ipython-input-1897159303.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [9]:
# chatbot itself

template = """
You are a helpful scientific assistant.
Use ONLY the provided research context to answer the question.

Context:
{context}

Question:
{question}

Answer concisely and clearly.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

llm = ChatOpenAI(
    model_name="gpt-4-turbo",
    temperature=0,
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    # simplest way to stuff retrieved chunks into the prompt
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [13]:
# code for querying is here!!

query = "What is typical harvesting routines in Minnesota?"
result = qa_chain.run(query)
print("💬✨", result, "✨")

💬✨ In Minnesota, the typical harvesting routine for strawberries involves harvesting every other day, although this may increase to daily harvesting if there are hot weather conditions or infestations of spotted wing drosophila (SWD). ✨
